In [1]:
import numpy as np
import pandas as pd

In [28]:
raw_data = pd.read_csv('./data/bike_sharing_demand.csv', header = 0, sep = ',')

In [29]:
#применение действия к столбцу
raw_data.datetime = raw_data.datetime.apply(pd.to_datetime)
#удаление столбцов
raw_data = raw_data.drop(['datetime'], axis = 1)
#заполнение нуллов
raw_data.fillna('', inplace=True)

In [30]:
y = raw_data['count'].values
X = raw_data.drop(['count'], axis = 1)

In [31]:
#Если всё получается просто замечательно, стоит проверить признаки
print(np.all(X.registered + X.casual == y))
X.drop(['casual', 'registered'], axis = 1, inplace = True)

True


In [37]:
# Нужно граммотно обращаться с категориальными, бинарыми и вещественными признаками
import sklearn.preprocessing
#определим эти признаки
binary_data_columns = ['holiday', 'workingday']
binary_data_indices = np.array([(column in binary_data_columns) for column in X.columns], dtype = bool)
categorical_data_columns = ['season', 'weather'] 
categorical_data_indices = np.array([(column in categorical_data_columns) for column in X.columns], dtype = bool)
numeric_data_columns = ['temp', 'atemp', 'humidity', 'windspeed']
numeric_data_indices = np.array([(column in numeric_data_columns) for column in X.columns], dtype = bool)

In [25]:
# Делает из функции транформер с методом fit (для использования в пайплайне)
transformer = sklearn.preprocessing.FunctionTransformer(lambda data: data[:, binary_data_indices])

FunctionTransformer(accept_sparse=False, check_inverse=True,
          func=<function <lambda> at 0x000002B2065552F0>, inv_kw_args=None,
          inverse_func=None, kw_args=None, pass_y='deprecated',
          validate=None)

In [34]:
#Масштабирование признаков
scaler = sklearn.preprocessing.StandardScaler()
scaler.fit(X[numeric_data_columns], y)
scaled_X = scaler.transform(X[numeric_data_columns])

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.


In [38]:
# обработка категориальных признаков, превращением их в вещественные
encoder = sklearn.preprocessing.OneHotEncoder(handle_unknown = 'ignore')
encoder.fit(X[categorical_data_columns], y)
encoded_X = encoder.transform(X[categorical_data_columns])

In [40]:
# Можно полиномиально преобразовать признаки (существенно увеличивает пространство признаков)
transform = sklearn.preprocessing.PolynomialFeatures(2)
train_poly = transform.fit_transform(scaled_X)
test_poly = transform.transform(scaled_X) # не забываем преобразовать также признаки в тесте

## Обработка текстов

In [1]:
# возьмём один из стандартных датасетов
from sklearn.datasets import fetch_20newsgroups
# возьмём из него небольшую подвыборку
simple_dataset = fetch_20newsgroups(
    subset='train', 
    categories=['comp.sys.mac.hardware', 'soc.religion.christian', 'rec.sport.hockey'])

In [3]:
#обработка признаков
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer = TfidfVectorizer(max_df=500, min_df=10)
matrix = vectorizer.fit_transform(simple_dataset.data)

vectorizer.get_feature_names() #посмотреть вектор фичей